In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import random
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90., high=90., size=2000)
lngs = np.random.uniform(low=-180., high=180., size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

726

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14d496981f3ab4ab4fbc15adae63ab96


In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]("{:.0f}".format)
        city_clouds = city_weather["clouds"]["all"]("{:.0f}".format)
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | carnarvon
City not found. Skipping...
Processing Record 2 of Set 1 | vao
City not found. Skipping...
Processing Record 3 of Set 1 | port moresby
City not found. Skipping...
Processing Record 4 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 5 of Set 1 | cabo san lucas
City not found. Skipping...
Processing Record 6 of Set 1 | eureka
City not found. Skipping...
Processing Record 7 of Set 1 | witu
City not found. Skipping...
Processing Record 8 of Set 1 | ushuaia
City not found. Skipping...
Processing Record 9 of Set 1 | leningradskiy
City not found. Skipping...
Processing Record 10 of Set 1 | dicabisagan
City not found. Skipping...
Processing Record 11 of Set 1 | hobart
City not found. Skipping...
Processing Record 12 of Set 1 | urdzhar
City not found. Skipping...
Processing Record 13 of Set 1 | new norfolk
City not found. Skipping...
Processing Record 14 of Set 1 | yellowkni

City not found. Skipping...
Processing Record 20 of Set 3 | karakol
City not found. Skipping...
Processing Record 21 of Set 3 | tuatapere
City not found. Skipping...
Processing Record 22 of Set 3 | barentsburg
City not found. Skipping...
Processing Record 23 of Set 3 | touros
City not found. Skipping...
Processing Record 24 of Set 3 | kyabram
City not found. Skipping...
Processing Record 25 of Set 3 | flinders
City not found. Skipping...
Processing Record 26 of Set 3 | meyungs
City not found. Skipping...
Processing Record 27 of Set 3 | bandarbeyla
City not found. Skipping...
Processing Record 28 of Set 3 | awjilah
City not found. Skipping...
Processing Record 29 of Set 3 | bela
City not found. Skipping...
Processing Record 30 of Set 3 | puerto narino
City not found. Skipping...
Processing Record 31 of Set 3 | barra da estiva
City not found. Skipping...
Processing Record 32 of Set 3 | durango
City not found. Skipping...
Processing Record 33 of Set 3 | kawalu
City not found. Skipping...


City not found. Skipping...
Processing Record 39 of Set 5 | utica
City not found. Skipping...
Processing Record 40 of Set 5 | bijie
City not found. Skipping...
Processing Record 41 of Set 5 | bandar-e torkaman
City not found. Skipping...
Processing Record 42 of Set 5 | bolungarvik
City not found. Skipping...
Processing Record 43 of Set 5 | fort nelson
City not found. Skipping...
Processing Record 44 of Set 5 | nouadhibou
City not found. Skipping...
Processing Record 45 of Set 5 | alad
City not found. Skipping...
Processing Record 46 of Set 5 | yingshang
City not found. Skipping...
Processing Record 47 of Set 5 | mutsamudu
City not found. Skipping...
Processing Record 48 of Set 5 | mahajanga
City not found. Skipping...
Processing Record 49 of Set 5 | hithadhoo
City not found. Skipping...
Processing Record 50 of Set 5 | brae
City not found. Skipping...
Processing Record 1 of Set 6 | morgan city
City not found. Skipping...
Processing Record 2 of Set 6 | pangnirtung
City not found. Skippin

City not found. Skipping...
Processing Record 8 of Set 8 | puerto escondido
City not found. Skipping...
Processing Record 9 of Set 8 | barra do garcas
City not found. Skipping...
Processing Record 10 of Set 8 | savannah bight
City not found. Skipping...
Processing Record 11 of Set 8 | reconquista
City not found. Skipping...
Processing Record 12 of Set 8 | san ignacio
City not found. Skipping...
Processing Record 13 of Set 8 | puta
City not found. Skipping...
Processing Record 14 of Set 8 | puerto ayacucho
City not found. Skipping...
Processing Record 15 of Set 8 | san miguel
City not found. Skipping...
Processing Record 16 of Set 8 | billings
City not found. Skipping...
Processing Record 17 of Set 8 | tiznit
City not found. Skipping...
Processing Record 18 of Set 8 | hualmay
City not found. Skipping...
Processing Record 19 of Set 8 | dalvik
City not found. Skipping...
Processing Record 20 of Set 8 | payo
City not found. Skipping...
Processing Record 21 of Set 8 | mufulira
City not foun

City not found. Skipping...
Processing Record 26 of Set 10 | inhambane
City not found. Skipping...
Processing Record 27 of Set 10 | ruatoria
City not found. Skipping...
Processing Record 28 of Set 10 | callaway
City not found. Skipping...
Processing Record 29 of Set 10 | akureyri
City not found. Skipping...
Processing Record 30 of Set 10 | myitkyina
City not found. Skipping...
Processing Record 31 of Set 10 | puerto madryn
City not found. Skipping...
Processing Record 32 of Set 10 | jinchang
City not found. Skipping...
Processing Record 33 of Set 10 | north platte
City not found. Skipping...
Processing Record 34 of Set 10 | dombarovskiy
City not found. Skipping...
Processing Record 35 of Set 10 | lamu
City not found. Skipping...
Processing Record 36 of Set 10 | chippewa falls
City not found. Skipping...
Processing Record 37 of Set 10 | kralendijk
City not found. Skipping...
Processing Record 38 of Set 10 | sangar
City not found. Skipping...
Processing Record 39 of Set 10 | atar
City no

City not found. Skipping...
Processing Record 43 of Set 12 | bridlington
City not found. Skipping...
Processing Record 44 of Set 12 | nagapattinam
City not found. Skipping...
Processing Record 45 of Set 12 | bambanglipuro
City not found. Skipping...
Processing Record 46 of Set 12 | sharjah
City not found. Skipping...
Processing Record 47 of Set 12 | huarmey
City not found. Skipping...
Processing Record 48 of Set 12 | harnosand
City not found. Skipping...
Processing Record 49 of Set 12 | krasnoselkup
City not found. Skipping...
Processing Record 50 of Set 12 | tygda
City not found. Skipping...
Processing Record 1 of Set 13 | lere
City not found. Skipping...
Processing Record 2 of Set 13 | san benito abad
City not found. Skipping...
Processing Record 3 of Set 13 | labutta
City not found. Skipping...
Processing Record 4 of Set 13 | karaton
City not found. Skipping...
Processing Record 5 of Set 13 | mizdah
City not found. Skipping...
Processing Record 6 of Set 13 | bulaevo
City not found. 

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
city_data_df.to_csv('WeatherPy_Database.csv')